# 웹캠

In [8]:
import cv2

def Main():
    camera = cv2.VideoCapture(source)
    
    while(True):
        
        ret, frame = camera.read()
        
        if not ret:
            break
        
        cv2.imshow("Webcam",frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    camera.release();
    cv2.destroyAllWindows()
    

In [9]:
source = "data/manv.mp4"

if __name__ == '__main__' :
    Main()

# 얼굴감지와 웹캠

In [1]:
import cv2
import numpy as np
import openvino as ov

## 모델 부르기

In [2]:
core = ov.Core()
model_face = core.read_model(model='models/face/face-detection-adas-0001.xml') 
compiled_model_face = core.compile_model (model = model_face, device_name="CPU")

input_layer_face = compiled_model_face.input(0) 
output_layer_face = compiled_model_face.output(0)

print("Input shape:", input_layer_face.shape) 
print("Output shape:", output_layer_face.shape)

model_emo = core.read_model(model='models/emotion/emotions-recognition-retail-0003.xml') 
compiled_model_emo = core.compile_model (model = model_emo, device_name="CPU")

input_layer_emo = compiled_model_emo.input(0) 
output_layer_emo = compiled_model_emo.output(0)

print("Input shape:", input_layer_emo.shape) 
print("Output shape:", output_layer_emo.shape)

model_ag = core.read_model(model='models/agegender/age-gender-recognition-retail-0013.xml') 
compiled_model_ag = core.compile_model (model = model_ag, device_name="CPU")

input_layer_ag = compiled_model_ag.input(0) 
output_layer_ag = compiled_model_ag.output

print("Input shape:", input_layer_ag.shape) 
print("Output shape:", output_layer_ag)

Input shape: [1,3,384,672]
Output shape: [1,1,200,7]
Input shape: [1,3,64,64]
Output shape: [1,5,1,1]
Input shape: [1,3,62,62]
Output shape: <bound method PyCapsule.output of <CompiledModel:
inputs[
<ConstOutput: names[data] shape[1,3,62,62] type: f32>
]
outputs[
<ConstOutput: names[prob] shape[1,2,1,1] type: f32>,
<ConstOutput: names[age_conv3, fc3_a] shape[1,1,1,1] type: f32>
]>>


### pre process new input data

In [3]:
import numpy as np
def preprocess(frame, input_layer):
    N, input_channels, input_height, input_width = input_layer.shape
    
    resized_frame = cv2.resize(frame, (input_width, input_height))
    transposed_frame = resized_frame.transpose(2,0,1)
    input_frame = np. expand_dims(transposed_frame, 0)
    
    return input_frame
    

### postprocess the results

In [4]:
def find_faceboxes(frame, results, confidence_threshold):
    results = results.squeeze()
    
    scores = results[:,2]
    boxes = results[:,-4:]
    
    face_boxes = boxes[scores >= confidence_threshold]
    scores = scores[scores >= confidence_threshold]
    
    frame_h, frame_w, frame_channels = frame.shape
    face_boxes = face_boxes*np.array([frame_w, frame_h, frame_w, frame_h])
    face_boxes = face_boxes.astype(np.int64)
    
    return face_boxes, scores

### draw_age_gender

In [5]:
def draw_age_gender_emotion(face_boxes, frame):
    
    show_frame = frame.copy()
    EMOTION_NAMES = ['neutral', 'happy', 'sad', 'surprise', 'anger']
    for i in range(len(face_boxes)):
        
            xmin, ymin, xmax, ymax = face_boxes[i]
            face = frame[ymin:ymax, xmin:xmax]
            
            #---emotion---
            input_frame = preprocess(face, input_layer_emo)
            results_emo = compiled_model_emo([input_frame])[output_layer_emo]
        
            results_emo = results_emo.squeeze()
            index = np.argmax(results_emo)
            #---emotion---
            
            
            # --- age gender ---
            input_frame_ag = preprocess(face, input_layer_ag)
            results_ag = compiled_model_ag([input_frame_ag])
            age, gender = results_ag[1], results_ag[0]
            
            age = np.squeeze(age)
            age = int(age*100)
            
            gender = np.squeeze(gender)
            
            if (gender[0] >= 0.65):
                gender = "female"
                box_color = (255,153,255)
            elif (gender[1] >= 0.65):
                gender = "male"
                box_color = (255,204,154)
            else :
                gender = "unknown"
                box_color = (0,0,0)
                
            
            #print("Age:",age)
            #print("Gender:",gender)
            #print(results_ag)
            # --- age gender ---
            
            fontScale = frame.shape[1]/750
           
            text = gender + ' ' + str(age) + ' ' + EMOTION_NAMES[index]
            cv2.putText(show_frame,text,(xmin,ymin), cv2.FONT_HERSHEY_SIMPLEX, fontScale, box_color ,4)
            cv2.rectangle(img=show_frame, pt1=(xmin,ymin), pt2=(xmax,ymax), color= box_color ,thickness=4)
    
    return show_frame
            

# Run inference

In [6]:
import cv2

def Main():
    camera = cv2.VideoCapture(source)
    
    while(True):
        
        ret, frame = camera.read()
        
        if not ret:
            break
            
        input_frame = preprocess(frame, input_layer_face)
        results = compiled_model_face([input_frame])[output_layer_face]

        
        face_boxes, scores = find_faceboxes(frame, results, confidence_threshold)
        show_frame = draw_age_gender_emotion(face_boxes, frame)
        
        cv2.imshow("Webcam",show_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    camera.release();
    cv2.destroyAllWindows()

confidence_threshold = .95
source = "data/manv.mp4"

if __name__ == '__main__' :
    Main()
    

In [7]:
import cv2

def Main():
    camera = cv2.VideoCapture(0)  
    while True:
        ret, frame = camera.read()
        
        
        if not ret:
            break
            
        input_frame = preprocess(frame, input_layer_face)
        results = compiled_model_face([input_frame])[output_layer_face]
        
        face_boxes, scores = find_faceboxes(frame, results, confidence_threshold)
        show_frame = draw_age_gender_emotion(face_boxes, frame)
        
        cv2.imshow("Webcam", show_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    camera.release()
    cv2.destroyAllWindows()

confidence_threshold = .95
source = 0 

if __name__ == '__main__':
    Main()
